## 集合类型

In [13]:
a='helloworld'
s=" ".join([a,"jack"])
print(s)

helloworld jack


In [19]:
evens = [i for i in range(10) if i % 2 == 0]
print(evens)

[0, 2, 4, 6, 8]


In [20]:
l=['one','two','three']
for index,value in enumerate(l):
    print(index,":",value)

0 : one
1 : two
2 : three


In [21]:
for item in zip([1,2,3],[4,5,6]):
    print(item)

(1, 4)
(2, 5)
(3, 6)


In [23]:
for item in zip(*zip([1,2,3],[4,5,6])):
    print(item)

(1, 2, 3)
(4, 5, 6)


In [24]:
first,second, *rest = 0,1,2,3
print(first)
print(second)
print(rest)

0
1
[2, 3]


In [25]:
first, *inner, last = 0,1,2,3,4,5
print(first)
print(inner)
print(last)

0
[1, 2, 3, 4]
5


In [27]:
squares = {num:num**2 for num in range(10)}
print(squares)

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16, 5: 25, 6: 36, 7: 49, 8: 64, 9: 81}


In [30]:
squares.items()

dict_items([(0, 0), (1, 1), (2, 4), (3, 9), (4, 16), (5, 25), (6, 36), (7, 49), (8, 64), (9, 81)])

In [32]:
{str(number): None for number in range(5)}.keys()

dict_keys(['0', '1', '2', '3', '4'])

In [36]:
s1=set([a,'b'])
print(s1)

{'b', 'helloworld'}


In [37]:
set(a)

{'d', 'e', 'h', 'l', 'o', 'r', 'w'}

# 高级语法

## 迭代器

In [41]:
i=iter('hello')
next(i)


'h'

In [59]:
class CountDown:
    def __init__(self,step):
        self.step = step
        
    def __next__(self):
        if self.step <=0:
            raise StopIteration
        self.step -= 1
        return self.step
    
    def __iter__(self):
        return self



In [60]:
for i in CountDown(5):
    print(i)

4
3
2
1
0


## yield语句

In [1]:
def fibonacci():
    a,b = 0,1
    while True:
        yield b
        a,b = b, a+b
        

In [2]:
fib = fibonacci()
next(fib)

1

In [13]:
def power(values):
    for value in values:
        print("powering %s " % value)
        yield value
        
def adder(values):
    for value in values:
        print('adding to %s' % value)
        if value % 2 == 0:
            yield value + 3
        else:
            yield value + 2

In [14]:
elements = [1,4,7,9,12,19]
results = adder(power(elements))

In [15]:
next(results)

powering 1 
adding to 1


3

In [23]:
def psychologist():
    print('Please tell me your problems')
    while True:
        answer = (yield)
        if answer is not None:
            if answer.endswith('?'):
                print("Don't ask yourself too much questions")
            elif 'good' in answer:
                print("Ahh that's good,go on")
            elif 'bad' in answer:
                print("Don't be so negative")

In [24]:
free = psychologist()
next(free)

Please tell me your problems


In [25]:
free.send("I'm bad")

Don't be so negative


In [26]:
free.send("Why I shouldn't?")

Don't ask yourself too much questions


In [27]:
free.send("ok, then i should find what is good for me")

Ahh that's good,go on


## 装饰器

In [28]:
class WithoutDecorators:
    def some_static_method():
        print("this is static method")
    some_static_method = staticmethod(some_static_method)
    
    def some_class_method(cls):
        print("this is class methon")
    some_class_method = classmethod(some_class_method)
    

In [ ]:
class WithDecorators:
    @staticmethod
    def some_static_method():
        print("this is static method1")
        
    @classmethod
    def some_class_method(cls):
        print("this is class method1")

In [ ]:
def mydecorator(function):
   def wrapped(*args, **kwargs):
       # 在调用原始函数之前，做点什么
       result = function(*args, **kwargs)
       # 在函数调用之后，做点什么，
       # 并返回结果
       return result
   # 返回wrapper作为装饰函数
   return wrapped

In [29]:
def repeat(number=3):
    """多次重复执行装饰函数
    """
    def actual_decorator(function):
        def wrapper(*args,**kwargs):
            result = None
            for _ in range(number):
                result = function(*args, **kwargs)
            return result
        return wrapper
    return actual_decorator

In [30]:
@repeat(2)
def foo():
    print("foo")
    

In [31]:
foo()

foo
foo


In [32]:
@repeat()
def bar():
    print("bar")

In [33]:
bar()

bar
bar
bar


In [34]:
def dummy_decorator(function):
    def wrapped(*args, **kwargs):
        """包装函数内部文档"""
        return function(*args,**kwargs)
    return wrapped

In [36]:
@dummy_decorator
def function_with_important_docstring():
    """这是我们想要保存的重要文档字符串"""

In [37]:
function_with_important_docstring.__name__

'wrapped'

In [38]:
function_with_important_docstring.__doc__

'包装函数内部文档'

In [40]:
rpc_info = {}

def xmlrpc(in_=(), out=(type(None),)):
    def _xmlrpc(function):
        #注册签名
        func_name = function.__name__
        rpc_info[func_name] = (in_,out)
        
        def _check_types(elements, types):
            """用来检查类型的子函数。"""
            if len(elements) != len(types):
                raise TypeError('argument count is wrong')
            typed = enumerate(zip(elements,types))
            for index, couple in typed:
                arg, of_the_right_type = couple
                if isinstance(arg, of_the_right_type):
                    continue
                raise TypeError('arg #%d should be %s' % (index, of_the_right_type))
                
        # 包装过的函数
        def __xmlrpc(*args): #没有允许的关键词
            #检查输入的内容
            checkable_args = args[1:]
            _check_types(checkable_args, in_)
            #运行函数
            res = function(*args)
            # 检查输出的内容
            if not type(res) in (tuple, list):
                checkable_res = (res,)
            else:
                checkable_res = res
            _check_types(checkable_res, out)
            
            # 函数及类型检查成功
            return res
        return __xmlrpc
    return _xmlrpc

In [41]:
class RPCView:
    @xmlrpc((int,int)) # two int -> None
    def meth1(self, int1, int2):
        print('received %d and %d' % (int1,int2))
        
    @xmlrpc((str,),(int,)) # string -> int
    def meth2(self, phrase):
        print('received %s' % phrase)
        return 12

In [42]:
rpc_info

{'meth1': ((int, int), (NoneType,)), 'meth2': ((str,), (int,))}

In [43]:
my = RPCView()
my.meth1(1,2)

received 1 and 2


In [45]:
my.meth2("hello")

received hello


12

In [47]:
import time
import hashlib
import pickle

cache = {}

def is_obsolete(entry, duration):
   return time.time() - entry['time'] > duration

def compute_key(function, args, kw):
   key = pickle.dumps((function.__name__, args, kw))
   return hashlib.sha1(key).hexdigest()

def memoize(duration=10):
   def _memoize(function):
       def __memoize(*args, **kw):
           key = compute_key(function, args, kw)

           # 是否已经拥有它了？
           if (key in cache and
               not is_obsolete(cache[key], duration)):
               print('we got a winner')
               return cache[key]['value']
           # 计算
           result = function(*args, **kw)
           # 保存结果
           cache[key] = {
               'value': result,
               'time': time.time()
           }
           return result
       return __memoize
   return _memoize

In [48]:
@memoize()
def very_very_very_complex_stuff(a,b):
    # 如果执行时计算机死机，请中止程序
    return a + b


In [55]:
very_very_very_complex_stuff(2,5)

7

In [59]:
very_very_very_complex_stuff(2,5)

7

In [60]:
cache

{'47833c15f4beeda0b7b0177b98798073e819e499': {'value': 5,
  'time': 1543756147.930739},
 'c4ba025dbed84bd8eb75d4beacf6900922117068': {'value': 4,
  'time': 1543756199.291903},
 '2146d952b9dca95c4fee021d7b7f26464c3db337': {'value': 7,
  'time': 1543756369.251295}}

In [58]:
cache

{'47833c15f4beeda0b7b0177b98798073e819e499': {'value': 5,
  'time': 1543756147.930739},
 'c4ba025dbed84bd8eb75d4beacf6900922117068': {'value': 4,
  'time': 1543756199.291903},
 '2146d952b9dca95c4fee021d7b7f26464c3db337': {'value': 7,
  'time': 1543756306.41521}}

### 代理

In [61]:
class User(object):
   def __init__(self, roles):
       self.roles = roles

class Unauthorized(Exception):
   pass

def protect(role):
   def _protect(function):
       def __protect(*args, **kw):
           user = globals().get('user')
           if user is None or role not in user.roles:
               raise Unauthorized("I won't tell you")
           return function(*args, **kw)
       return __protect
   return _protect

In [62]:
tarek = User(('admin', 'user'))
bill = User(('user',))

In [63]:
class MySecrets(object):
    @protect('admin')
    def waffle_recipe(self):
        print("use tons of butter")

In [64]:
these_are = MySecrets()

In [65]:
user = tarek

In [66]:
these_are.waffle_recipe()

use tons of butter


In [67]:
user = bill
these_are.waffle_recipe()

Unauthorized: I won't tell you

### 上下文提供者

In [68]:
from threading import RLock
lock = RLock()

def synchronized(function):
   def _synchronized(*args, **kw):
       lock.acquire()
       try:
           return function(*args, **kw)
       finally:
           lock.release()
       return _synchronized

@synchronized
def thread_safe():  # 确保锁定资源
   pass

### with语句

In [69]:
hosts = open("/etc/hosts")
try:
    for line in hosts:
        if line.startswith("#"):
            continue
        print(line.strip())
finally:
    hosts.close()

127.0.0.1	localhost
255.255.255.255	broadcasthost
52.84.246.72    d3c33hcgiwev3.cloudfront.net
::1             localhost

127.0.0.1 windows10.microdone.cn



In [70]:
with open("/etc/hosts") as hosts:
    for line in hosts:
        if line.startswith("#"):
            continue
        print(line.strip())

127.0.0.1	localhost
255.255.255.255	broadcasthost
52.84.246.72    d3c33hcgiwev3.cloudfront.net
::1             localhost

127.0.0.1 windows10.microdone.cn



In [72]:
for n in range(3):
    break
else:
    print("break")

In [73]:
def f(ham:str, eggs:str = 'eggs') -> str:
    pass


In [74]:
print(f.__annotations__)

{'ham': <class 'str'>, 'eggs': <class 'str'>, 'return': <class 'str'>}


In [88]:
class Mama():
    def says(slef):
        print("do your homework")
        
class Sister(Mama):
    def says(self):
        super().says()
        print("and clean your bedroom")

In [89]:
anita=Sister()
anita.says()

do your homework
and clean your bedroom


In [90]:
super(anita.__class__, anita).says()

do your homework


In [102]:
class A:
   def __init__(self):
       print("A", end=" ")
       super(A).__init__()

class B:
   def __init__(self):
       print("B", end=" ")
       super().__init__()

class C(A, B):
   def __init__(self):
       print("C", end=" ")
       A.__init__(self)
       #B.__init__(self)

In [103]:
C()

C A 